In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import tensorflow as tf
print(os.listdir("../input"))
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4"

# Any results you write to the current directory are saved as output.

In [ ]:
X_train = pd.read_csv('../input/X_train.csv')
X_test= pd.read_csv('../input/X_test.csv')
y_train=pd.read_csv('../input/y_train.csv')

In [ ]:
X_train.series_id.groupby(y_train['surface']).describe()

In [ ]:
y_train.groupby(y_train['surface']).count()

In [ ]:
'''y_train['surface']=y_train['surface'].apply(lambda x:'hard_tiles' if x=='hard_tiles_large_space' else x)
y_train['surface']=y_train['surface'].apply(lambda x:'concrete' if x=='fine_concrete' else x)
y_train.groupby(y_train['surface']).count()'''

In [ ]:
one_hot=pd.get_dummies(y_train['surface'], prefix=None)
y_train=pd.concat([y_train,one_hot],axis=1)
y_train.head()
train_merged=pd.merge(X_train,y_train)

In [ ]:
x_train=train_merged.drop('surface',axis=1)
x_train=x_train.drop(['row_id','measurement_number'],axis=1)
x_train=x_train.drop('series_id',axis=1)
x_test=X_test.drop(['row_id','measurement_number'],axis=1)
series_id=x_test['series_id']
x_test=x_test.drop('series_id',axis=1)
print(x_test.shape)
print(x_train.shape)
x_train.head()
x_train=x_train.sample(n=x_train.shape[0],random_state=1)
x_train.head()

In [ ]:
labels=['carpet','concrete','fine_concrete','hard_tiles','hard_tiles_large_space','soft_pvc','soft_tiles','tiled','wood']
label_train=pd.DataFrame()
for i in labels:
    label_train=pd.concat([label_train,x_train[i]],axis=1)
x_train=x_train.drop(labels,axis=1)
print(x_test.shape)
x_train.head()

In [ ]:
label_train.head()

In [ ]:
x_train=x_train.drop('group_id',axis=1)
x_train.describe()
z=np.abs(stats.zscore(x_train))
row,column=np.where(z>3)
for i,j in zip(row,column):
    z[i][j]=np.median(z[i])

In [ ]:
x_train=pd.DataFrame(z,columns=x_train.columns,index=x_train.index)

In [ ]:
x_train.describe()
x_train.head()

In [ ]:
print(x_train.shape,label_train.shape)
X_split_train,X_split_test,Y_split_train,Y_split_test=train_test_split(x_train,label_train,stratify=label_train,train_size=0.7,random_state=0)

In [ ]:
X_split_train.head()

In [ ]:
scaler=MinMaxScaler()
X_transformed_train=pd.DataFrame(scaler.fit_transform(X_split_train),columns=X_split_train.columns,index=X_split_train.index)
X_transformed_test=pd.DataFrame(scaler.transform(X_split_test),columns=X_split_test.columns,index=X_split_test.index)
X_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns,index=x_test.index)

In [ ]:
X_transformed_train.head()
#X_test.describe()

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(X_transformed_train.corr(), annot=True, cmap='coolwarm')
correlation=X_transformed_train.corr()

In [ ]:
upper = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] >= 0.95) or any(upper[column] <= -0.95)]
print(to_drop)

In [ ]:
X_transformed_train=X_transformed_train.drop(to_drop,axis=1)

In [ ]:
X_transformed_test=X_transformed_test.drop(to_drop,axis=1)
X_test=X_test.drop(to_drop,axis=1)

In [ ]:
Y_split_train.head()

In [ ]:
learning_rate=0.001
epochs=1000
batch_size=512
k=64

x=tf.placeholder(tf.float32,[None,8])
y=tf.placeholder(tf.float32,[None,9])

w1=tf.Variable(tf.random_normal([8,30]),name='W1')
b1=tf.Variable(tf.random_normal([30]),name='B1')
w2=tf.Variable(tf.random_normal([30,20]),name='W2')
b2=tf.Variable(tf.random_normal([20]),name='B2')
w3=tf.Variable(tf.random_normal([20,9]),name='W3')
b3=tf.Variable(tf.random_normal([9]),name='B3')
'''w4=tf.Variable(tf.random_normal([7,20]),name='W4')
b4=tf.Variable(tf.random_normal([20]),name='B4')
w5=tf.Variable(tf.random_normal([20,20]),name='W5')
b5=tf.Variable(tf.random_normal([20]),name='B5')
w6=tf.Variable(tf.random_normal([20,7]),name='W6')
b6=tf.Variable(tf.random_normal([7]),name='B6')'''
'''with tf.device("/gpu:3"):
    w4=tf.Variable(tf.random_normal([15,7]),name='W4')
    b4=tf.Variable(tf.random_normal([7]),name='B4')'''
'''with tf.device("/gpu:4"):
    w5=tf.Variable(tf.random_normal([15,7]),name='W5')
    b5=tf.Variable(tf.random_normal([7]),name='B5')'''

In [ ]:
hidden_out1=tf.add(tf.matmul(x,w1),b1)
hidden_out1=tf.nn.sigmoid(hidden_out1)
#drop_out1=tf.nn.dropout(hidden_out1,0.2)
hidden_out2=tf.add(tf.matmul(hidden_out1,w2),b2)
#hidden_out2=tf.add(tf.matmul(drop_out1,w2),b2)
hidden_out2=tf.nn.sigmoid(hidden_out2)
#drop_out2=tf.nn.dropout(hidden_out2,0.2)
'''hidden_out3=tf.add(tf.matmul(hidden_out2,w3),b3)
hidden_out3=tf.nn.relu(hidden_out3)
hidden_out4=tf.add(tf.matmul(hidden_out3,w4),b4)
hidden_out4=tf.nn.relu(hidden_out4)
hidden_out5=tf.add(tf.matmul(hidden_out4,w5),b5)
hidden_out5=tf.nn.relu(hidden_out5)'''

In [ ]:
#y_=tf.add(tf.matmul(drop_out2,w3),b3)
y_=tf.add(tf.matmul(hidden_out2,w3),b3)
y_=tf.nn.softmax(y_)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_, labels=y))

In [ ]:
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
init=tf.global_variables_initializer()

correct_prediction=tf.equal(tf.argmax(y_,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
'''def batch_maker(batch_size,data_len,data):
    length=data_len
    list_rng=random.sample(length,batch_size)
    print(list_rng)'''

In [ ]:
seed=10
rng=np.random.RandomState(seed)
def batch_maker(batch_size,data_len,data):
    #print("data=",data.head)
    batch_mask=rng.choice(data_len,batch_size)
    print("batch_mask",batch_mask.shape)
    batch_is=data.reindex([batch_mask])
    print("Batch is=",batch_is)
    return batch_is

In [ ]:
X_transformed_train.shape

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    accur=[]
    ep=[]
    count=0
    flag=0
    for epochs in range(epochs):
        new_cost=0
        total_batch=int(X_transformed_train.shape[0]/batch_size)
        print(total_batch)
        #X_transformed_train=X_transformed_train.sample(n=X_transformed_train.shape[0],random_state=1)
        #Y_split_train=Y_split_train.sample(n=Y_split_train.shape[0],random_state=1)
        #print("X_trans=",X_transformed_train.head())
        #print("Y_split=",Y_split_train.head())
        print("Total batch=",total_batch)
        for i in range(0,X_transformed_train.shape[0],batch_size):
            #batch_x=batch_maker(batch_size,X_transformed_train.shape[0],X_transformed_train)
            batch_x=X_transformed_train[i:i+batch_size]
            #print(batch_x.head())
            #print(batch_x.head())
            #batch_y=batch_maker(batch_size,Y_split_train.shape[0],Y_split_train)
            batch_y=Y_split_train[i:i+batch_size]
            #print(batch_y.head())
            #print(batch_y.head())
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            new_cost+=c
        average_cost=new_cost/total_batch
        print ("Epoch:", (epochs+1), "cost =", "{:.5f}".format(average_cost),"Learning Rate =","{:.5f}".format(learning_rate))
        accur.append(sess.run(accuracy,feed_dict={x:X_transformed_test,y:Y_split_test}))
        ep.append(epochs)
        plt.scatter(ep,accur)
        '''if epochs>0:
            if flag==1:
                learning_rate=old_learning_rate
                flag=0
            if abs(accur[epochs-1]-accur[epochs])<=0.001:
                count+=1
                if count==10:
                    old_learning_rate=learning_rate
                    learning_rate=learning_rate*5
                    count=0
                    flag=1'''
        #plt.show()
        print(accur[-1])
        X_transformed_train=X_transformed_train.append(X_transformed_test[0:k])
        Y_split_train=Y_split_train.append(Y_split_test[0:k])
        X_transformed_test=X_transformed_test.iloc[k:]
        Y_split_test=Y_split_test.iloc[k:]
        #print("Train feat shape",X_transformed_train.shape[0])
        #print("Train label shape",Y_split_train.shape[0])
        #print("Test feat shape",X_transformed_test.shape[0])
        #print("Train feat shape",Y_split_test.shape[0])
    print('Training Complete')
    predict=tf.argmax(y_,1)
    pred=predict.eval({x:X_test})
    print(pred)

In [ ]:
labels={'0':'carpet','1':'concrete','2':'fine_concrete','3':'hard_tiles','4':'hard_tiles_large_space','5':'soft_pvc','6':'soft_tiles','7':'tiled','8':'wood'}
submission={}
data={}
predict={}
final_prediction=[]
submission['series_id']=series_id
for i in pred:
    final_prediction.append(labels[str(i)])
submission['Class']=final_prediction
submission=pd.DataFrame(submission)
x=submission['series_id'].unique().tolist()
values=[]
index=[]
for i in x:
    count=np.zeros(9)
    y=submission.loc[submission.series_id==i]
    for j in y.Class:
        for key,value in labels.items():
            if value==j:
                count[int(key)]=count[int(key)]+1
    values.append(labels[str(np.argmax(count))])
    index.append(str(i))
    print(count)
    #predict.update(data)
data={'series_id':index,'surface':values}
predi=pd.DataFrame(data)
predi.head(20)
#Y=X.loc[0,'Class']
#Y[1]        

In [ ]:
predi.tail(20)

In [ ]:
#submission=pd.DataFrame(submission)
predi.to_csv('submission.csv',index=False)
predi.head()